# Penalty Comparison: fidelity vs sensitivity as a function of λ (sensitivity weight)

This notebook compares the effectiveness of the toggling-frame robustness objective for both multiplicative and additive errors. It's strucuted as follows:
* Imports
* Problem Setups: Here we compare the effect of the toggling frame robustness objective on additive and multiplicative errors without any variational states.
    * Base case: unitary smooth pulse problem w/o any robustness or sentivity objectives
    * Toggle Robustness: Optimize pulses for toggle-frame robustness
* Plot Fidelity vs. Error
* Plot Trajectories and Control pulses
* Pareto Frontiers of Fidelity vs. Robustness Penality

## Imports

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.develop(path="../../QuantumCollocation.jl")
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using Statistics
using CairoMakie
using NamedTrajectories
using TrajectoryIndexingUtils
using Random
using CSV
using Measures: mm

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
   Resolving package versions...
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`


## Problem Setups
Here, we first setup the quantum systems and solve for optimal pulses. There is the default smooth unitary pulse case and the toggle-fram robustness subject to three error models (additive error, mujltiplcative error, both).

#### Preliminary Variables

In [2]:
# Problem parameters
T = 20
Δt = 0.5
U_goal = GATES.X
H_drive = [PAULIS.X, PAULIS.Y]
rob_scale = 1 / 8.0
piccolo_opts = PiccoloOptions(verbose=false)
da_bound=Inf
sys = QuantumSystem(H_drive)

QuantumSystem: levels = 2, n_drives = 2

Toggle setup

In [3]:
n_drives = sys.n_drives
n_guesses = 4
a_guess = [randn(2, T) for _ in 1:n_guesses]
a_bounds = fill(1/Δt, n_drives)
da_bounds = fill(1.0, n_drives)
dda_bounds = fill(1.0, n_drives)
control_bounds = (a_bounds, da_bounds, dda_bounds)
def_seeds = []
add_seeds = []
mult_seeds = []
both_seeds = []

for i in 1:n_guesses
    def_traj = initialize_trajectory(
                    U_goal,
                    T,
                    Δt,
                    n_drives,
                    control_bounds;
                    a_guess=deepcopy(a_guess[i]),
                    system=sys
                )
    push!(def_seeds, def_traj)

    add_traj = initialize_trajectory(
                    U_goal,
                    T,
                    Δt,
                    n_drives,
                    control_bounds;
                    a_guess=deepcopy(a_guess[i]),
                    system=sys
                )

    push!(add_seeds, add_traj)

    mult_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                a_guess=deepcopy(a_guess[i]),
                system=sys
            )

    push!(mult_seeds, mult_traj)

    both_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                a_guess=deepcopy(a_guess[i]),
                system=sys
            )

    push!(both_seeds, both_traj)
end

Run all seeds for various weights (lambdas) using the toggle objective on only the additive error problem

In [4]:
sweep_rob_loss_λ = [λ for λ in 1e-2:5:100.0]

20-element Vector{Float64}:
  0.01
  5.01
 10.01
 15.01
 20.01
 25.01
 30.01
 35.01
 40.01
 45.01
 50.01
 55.01
 60.01
 65.01
 70.01
 75.01
 80.01
 85.01
 90.01
 95.01

In [ ]:
sweep_rob_loss_λ = [λ for λ in 1e-2:5:100.0]
n_seeds = n_guesses
n_lambdas = length(sweep_rob_loss_λ)
add_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
Hₑ_add = a -> PAULIS.X

for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)

        # Add problem
        add_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt;
            init_trajectory=deepcopy(add_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=Hₑ_add,
            Q_t=λ
        )
        solve!(add_prob; max_iter=300, print_level=5)
        add_probs[i, λ_idx] = add_prob
    end
end

    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.8.0.

Number of nonzeros in equality constraint Jacobian...:     2082
Number of nonzeros in inequality constraint Jacobian.:        0
Number of no

setup the same problems for the default, multiplcative, both error cases

In [ ]:
#default
default_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        defaults = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; init_trajectory=deepcopy(def_seeds[i]))
        solve!(defaults; max_iter=300, print_level=5)
        default_probs[i,λ_idx] = defaults
    end
end

In [ ]:
# multiplicative
X_drive = sys.H.H_drives[1]
Hₑ_mult = a -> a[1] * X_drive
mult_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        # Mult problem
        mult_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt;
            init_trajectory=deepcopy(mult_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=Hₑ_mult,
            Q_t=λ
        )
        solve!(mult_prob; max_iter=300, print_level=5)
        mult_probs[i, λ_idx] = mult_prob
    end
end

In [ ]:
# both
# both_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

# # Optimization loop with index counter

#         # Both problem
#         both_prob = UnitaryVariationalProblem(
#             varsys_both, U_goal, T, Δt;
#             init_trajectory=deepcopy(varsys_both_seeds[i]),
#             variational_scales=[rob_scale,rob_scale],
#             sensitive_times=[[T]],
#             piccolo_options=piccolo_opts,
#             Q_s=λ
#         )
#         solve!(both_prob; max_iter=200, print_level=5)
#         both_probs[i, λ_idx] = both_prob
#     end
# end

plot fid v err magnitude for the above problems

In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()

# Plot for each λ
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        # ax3 = Axis(f[3, 1], title="Both noises, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, H_drive)
        multiplicative_system = ε -> QuantumSystem((1 + ε) * H_drive)
        # both_system = ε -> QuantumSystem(ε * PAULIS.X, [(1 + ε) * H_drive])
        # Plot data structure: (matrix, label, color)
        plot_configs = [
            (default_probs, "default", colors[1]),
            (add_probs, "add rob", colors[2]),
            (mult_probs, "mult rob", colors[4])#,
            # (both_adj_probs, "both rob", colors[4])
        ]

        # Plot for both noise types
        for (probs_matrix, label, color) in plot_configs
            # Additive noise
            prob = probs_matrix[seed_idx, λ_idx]
            ys_add = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem(ε * PAULIS.X, H_drive)) for ε in εs]
            lines!(ax1, εs, ys_add, label=label, color=color, linestyle=:solid)

            # Multiplicative noise
            ys_mult = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem((1 + ε) * H_drive)) for ε in εs]
            lines!(ax2, εs, ys_mult, label=label, color=color, linestyle=:solid)

            # # Both noises
            # ys_both = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem(ε * PAULIS.X, (1 + ε) * H_drive)) for ε in εs]
            # lines!(ax3, εs, ys_both, label=label, color=color, linestyle=:solid)
        end

        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        # Legend(f[3, 2], ax3, position=:lb)

        display(f)
    end
end

In [ ]:
p1 = CairoMakie.plot(defaults.trajectory, [:a, :Ũ⃗])
p2 = CairoMakie.plot(add_prob.trajectory, [:a, :Ũ⃗])
p3 = CairoMakie.plot(mult_prob.trajectory, [:a, :Ũ⃗])
# p4 = CairoMakie.plot(both_prob.trajectory, [:a, :Ũ⃗])

display(p1)
display(p2)
display(p3)
# display(p4)

In [ ]:
# Detailed results table
display("\n=== Detailed Results Table ===")
for seed_idx in 1:n_seeds
    display("\nseed idx = $seed_idx:")
    display("Weight λ | Base Fidelity | Add Fidelity | Mult Fidelity | Both")
    display("-" ^ 40)
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        def_fid = unitary_rollout_fidelity(tog_defaults[seed_idx, λ_idx].trajectory, sys)
        add_fid = unitary_rollout_fidelity(add_probs[seed_idx, λ_idx].trajectory, varsys_add)
        mult_fid = unitary_rollout_fidelity(add_probs[seed_idx, λ_idx].trajectory, varsys_mult)
        # both_fid = unitary_rollout_fidelity(both_tog_probs[seed_idx, λ_idx].trajectory, varsys_both)
        display("$(lpad(round(λ, digits=4), 7)) | $(lpad(round(def_fid, digits=6), 12)) | $(lpad(round(add_fid, digits=6), 12)) | $(lpad(round(mult_fid, digits=6), 13)) |")# $(lpad(round(both_fid, digits=6), 12)) |")
    end
end

## Sensitivity Plots

### Robust

In [ ]:
additive_fidelities = Matrix{Any}(undef, n_seeds, n_lambdas)
multiplicative_fidelities = Matrix{Any}(undef, n_seeds, n_lambdas)
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        add_prob = add_probs[seed_idx, λ_idx]
        add_fid = unitary_rollout_fidelity(add_prob.trajectory, sys)
        additive_fidelities[seed_idx, λ_idx] = add_fid
        mult_prob = mult_probs[seed_idx, λ_idx]
        mult_fid = unitary_rollout_fidelity(mult_prob.trajectory, sys)
        multiplicative_fidelities[seed_idx, λ_idx] = mult_fid
    end
end

In [ ]:
Hₑ_add = a -> PAULIS.X
X_drive = sys.H.H_drives[1]
Hₑ_mult = a -> a[1] * X_drive

additive_obj = Matrix{Any}(undef, n_seeds, n_lambdas)
multiplicative_obj = Matrix{Any}(undef, n_seeds, n_lambdas)

for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        add_prob = add_probs[seed_idx, λ_idx]
        obj = FirstOrderObjective(Hₑ_add, add_prob.trajectory)
        Z_vec = vec(add_prob.trajectory)
        add_obj_val = 1/obj.L(Z_vec)
        additive_obj[seed_idx, λ_idx] = 1/add_obj_val

        mult_prob = mult_probs[seed_idx, λ_idx]
        obj = FirstOrderObjective(Hₑ_mult, mult_prob.trajectory)
        Z_vec = vec(mult_prob.trajectory)
        mult_obj_val = 1/obj.L(Z_vec)
        multiplicative_obj[seed_idx, λ_idx] = 1/mult_obj_val
    end
end

In [ ]:
gr(fmt = :png)

# y = 1 - fidelity, floored for log-scale
to_infidelity(F::AbstractVector; floor_eps=1e-12) = max.(1 .- F, floor_eps)

# utilities
safe_floor_pos(v, eps) = (isfinite(v) && v > 0) ? v : eps
function make_logsafe_x(xraw::AbstractVector; invert::Bool=false, eps_base::Float64=1e12)
    pos = filter(v -> isfinite(v) && v > 0, xraw)
    eps = isempty(pos) ? eps_base : 0.1 * minimum(pos)
    xpos = map(v -> safe_floor_pos(v, eps), xraw)
    if invert
        return 1.0 ./ xpos
    else
        return xpos
    end
end

function log_axis_limits(x::AbstractVector; loq=0.01, hiq=0.98)
    xpos = filter(>(0), x)
    if isempty(xpos)
        return (1e-8, 1.0)
    end
    xlo = try quantile(xpos, loq) catch; minimum(xpos) end
    xhi = try quantile(xpos, hiq) catch; maximum(xpos) end
    xlo = max(xlo, minimum(xpos))
    xhi = max(xhi, nextfloat(xlo))
    (10.0 ^ floor(log10(xlo)), 10.0 ^ ceil(log10(xhi)))
end

titles      = ["Add", "Mult"]
fids_by_cfg = [additive_fidelities,  multiplicative_fidelities]
obj_by_cfg  = [additive_obj,         multiplicative_obj]   # your arrays

invert_x = true  # keep your intent of using 1 ./ objective

for seed_idx in 1:n_seeds
    for (cfg_idx, title_name) in enumerate(titles)
        Ftbl = fids_by_cfg[cfg_idx]
        objs = obj_by_cfg[cfg_idx]

        # prepare data (log-safe)
        x_all = make_logsafe_x(vec(objs[seed_idx, :]); invert=invert_x)
        y_all = to_infidelity(vec(Ftbl[seed_idx, :]))
        λ_all = collect(sweep_rob_loss_λ)

        # limits
        xlo, xhi = log_axis_limits(x_all)
        ylo, yhi = log_axis_limits(y_all)

        p = Plots.scatter(
            x_all, y_all;
            zcolor = λ_all, color = :viridis,
            colorbar = true, colorbar_title = "λ", colorbar_position = :right,
            # use Measures for ALL margins so no type-mix:
            left_margin   = 4mm,
            right_margin  = 8mm,
            top_margin    = 4mm,
            bottom_margin = 4mm,
            xlabel = invert_x ? "1 / Robust Objective (log10)" : "Robust Objective (log10)",
            ylabel = "Infidelity 1 - Fidelity (log10)",
            title  = "$title_name (seed $seed_idx): Infidelity vs Objective",
            xscale = :log10, yscale = :log10,
            xlims = (xlo, xhi), ylims = (ylo, yhi),
            markersize = 5,
            grid = true, gridalpha = 0.3, gridcolor = :gray,
            legend = false,
            size = (900, 600),   # optional: consistent PNG size
        )

        fn = "infidelity_vs_objective_$(title_name)_seed$(seed_idx).png"
        savefig(p, fn)
        println("Saved $fn")
    end
end

In [ ]:
to_infidelity(F::AbstractArray; floor_eps=1e-12) = max.(1 .- F, floor_eps)
julia_show_me_the_plots_please = []
titles = ["Add", "Mult"]
fids_by_cfg = [additive_fidelities, multiplicative_fidelities]
obj_by_cfg = [additive_obj, multiplicative_obj]

for (cfg_idx, title_name) in enumerate(titles)
    Ftbl = fids_by_cfg[cfg_idx]      # size ≈ (n_seeds, n_pts)
    objs = obj_by_cfg[cfg_idx]       # size ≈ (n_seeds, n_pts)

    # Average across seeds (dimension 1)
    x_all = vec(mean(objs; dims=1))                      # mean robust objective per λ
    y_all = vec(mean(to_infidelity(Ftbl); dims=1))       # mean infidelity per λ
    λ_all = sweep_rob_loss_λ                             # one λ per point

    # Robust axis limits
    function log_axis_limits(x::AbstractVector; loq=0.01, hiq=0.98)
        xpos = filter(>(0), x)
        if isempty(xpos)
            return (1e-8, 1.0)
        end
        xlo = try quantile(xpos, loq) catch; minimum(xpos) end
        xhi = try quantile(xpos, hiq) catch; maximum(xpos) end
        xlo = max(xlo, minimum(xpos))
        xhi = max(xhi, nextfloat(xlo))
        (10.0 ^ floor(log10(xlo)), 10.0 ^ ceil(log10(xhi)))
    end

    xlo, xhi = log_axis_limits(x_all)
    ylo, yhi = log_axis_limits(y_all)

    p1 = Plots.scatter(
        x_all, y_all;
        zcolor = λ_all,
        color = :viridis,
        colorbar_title = "λ",
        colorbar = true,
        right_margin = 5mm,
        xlabel = "Robust Objective (log10)",
        ylabel = "Infidelity (log10)",
        title  = "$title_name: Mean Infidelity vs Mean Objective (across seeds)",
        xscale = :log10, yscale = :log10,
        xlims = (xlo, xhi), ylims = (ylo, yhi),
        markersize = 5,
        grid = true, gridalpha = 0.3, gridcolor = :gray,
        legend = false,
    )
    p1 = Plots.plot(p1, colorbar_position = :right)
    display(p1)
end

In [ ]:
obj_vals = zeros(n_seeds, n_lambdas)
Hₑ_add = a -> PAULIS.X
X_drive = sys.H.H_drives[1]
Hₑ_mult = a -> a[1] * X_drive

# fidelity storages per noise model
additive_fidelities       = zeros(n_seeds, n_lambdas)
multiplicative_fidelities = zeros(n_seeds, n_lambdas)

# Helper: safe infidelity for log plots
to_infidelity(F::AbstractVector; floor_eps=1e-12) = max.(1 .- F, floor_eps)

# Compute obj_vals and fidelities
configs = [
    (add_probs,  additive_fidelities,       Hₑ_add),
    (mult_probs, multiplicative_fidelities, Hₑ_mult)
]
titles = ["Add", "Mult"]

for (probs, fidelities, Hₑ) in configs
    println("----------------! New noise model !----------------")
    for seed_idx in 1:n_seeds
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            prob = probs[seed_idx, λ_idx]
            println("\nProcessing λ = $λ")

            obj = QuantumObjectives.FirstOrderObjective(Hₑ, prob.trajectory; Q_t=λ)

            Z_vec = vec(prob.trajectory)
            obj_vals[seed_idx, λ_idx] = (λ > 0 ? obj.L(Z_vec)/λ : NaN)

            # Robust rollout fidelity (against nominal system unless you want otherwise)
            fidelities[seed_idx, λ_idx] = unitary_rollout_fidelity(prob.trajectory, sys)
        end
    end
end

results = DataFrame(Seed=Int[], λ=Float64[], NoiseModel=String[],
                    ObjectiveValue=Float64[], Fidelity=Float64[])

for (cfg_idx, title_name) in enumerate(titles)
    Ftbl = (cfg_idx == 1) ? additive_fidelities : multiplicative_fidelities
    for seed_idx in 1:n_seeds, (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        push!(results, (
            seed_idx,
            λ,
            title_name,
            obj_vals[seed_idx, λ_idx],
            Ftbl[seed_idx, λ_idx],
        ))
    end
end

# Save the table (optional)
CSV.write("fidelity_vs_objective.csv", results)

# Show first rows inline
first(results, 100) |> display